In [1]:
import pandas as pd
from pathlib import Path
import random
import numpy as np

import VehicleSchedulingAlgorithm as vsa
import vsa_utils

# Load and split data

In [3]:
ipath = Path('../data/test_data')

vehicle_type = '_shuttle'

vehicle_capacity = 10
max_waiting_duration_min = 15
demand_duration = 60

%store -r max_waiting_duration_min
%store -r vehicle_capacity

In [4]:
zones = pd.read_csv(ipath / f'zones{vehicle_type}.csv', sep=';')
od = pd.read_csv(ipath / f'od_time{vehicle_type}.csv', sep=';').fillna(0).set_index(['from','to'])
distance_matrix = pd.read_csv(ipath / f'distance_matrix{vehicle_type}.csv', sep=';').fillna(0).set_index(['from','to'])

od.rename(columns={'0': 'volume'}, inplace=True)
od.dropna(inplace=True)

We are splitting the demand data into its part > 1 and part < 1:
- part < 1 is applied original algorithm without accounting for vehicle occupancy
- part > 1 is applied adapted algorithm accounting for vehicle occupancy

In [6]:
vsa_prep_full = vsa_utils.VSA_Preparation(
    zones, od[od['volume']>=1], distance_matrix,
    max_waiting_duration_min=max_waiting_duration_min,
    demand_duration=demand_duration,
    as_integers=True,
    vehicle_demand_to_integer=True,
    aggregate_method='logic',
    vehicle_capacity=vehicle_capacity
)

vsa_prep_decimal = vsa_utils.VSA_Preparation(
    zones, od[od['volume']<1], distance_matrix,
    max_waiting_duration_min=None,
    demand_duration=demand_duration,
    as_integers=False,
    vehicle_demand_to_integer=False,
    aggregate_method=None,
    vehicle_capacity=vehicle_capacity
)

c:\Users\rgoix\Documents\dev\quetzal_models\amod\VehicleScheduling\notebooks\vsa_utils.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.volumes.dropna(inplace=True)


# prepare

In [7]:
vsa_prep_full.discretize_aggregate()
distance_matrix_full, demand_matrix_full = vsa_prep_full.compute_vsa_inputs()

In [8]:
vsa_prep_decimal.discretize_aggregate()
distance_matrix_decimal, demand_matrix_decimal = vsa_prep_decimal.compute_vsa_inputs()

# run

In [9]:
scheduling_full = vsa.VSA(distance_matrix_full, demand_matrix_full, max_relocation_distance=None)
scheduling_full.solve()
scheduling_full.required_vehicles

528.0

In [10]:
scheduling_decimal = vsa.VSA(distance_matrix_decimal, demand_matrix_decimal, max_relocation_distance=None)
scheduling_decimal.solve()
scheduling_decimal.required_vehicles

52.627027940000005

In [11]:
n_vehs = (scheduling_decimal.required_vehicles, scheduling_full.required_vehicles)
%store n_vehs

Stored 'n_vehs' (tuple)
